In [6]:
import mysql.connector as SQLC
def search_db(query):
    cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG")
    cursor = cnx.cursor()
    cursor.execute("SELECT sequence FROM Gene")
    sequences = cursor.fetchall()
    cursor.close()
    return sequences
sequences=search_db("A")

In [53]:
#FINAL
class Blast:
    def __init__(self, w, seq):
        self.w = w
        self.seq = seq

    def query_map(self, query):
        query_map = {}
        for i in range(len(query) - self.w + 1):
            window = query[i: i + self.w]
            if window in query_map:
                query_map[window].append(i)
            else:
                query_map[window] = [i]
        return query_map

    def hits(self, query_map, sequence):
        hits = []
        for window, query_offsets in query_map.items():
            if window in sequence:
                seq_offset = sequence.find(window)
                while seq_offset != -1:
                    for query_offset in query_offsets:
                        hits.append((query_offset, seq_offset))
                    seq_offset = sequence.find(window, seq_offset + 1)
        return hits

    def extend_hit_dir(self, query, sequence, q_offset, s_offset, direction):
        count = 0
        matches = 0
        while q_offset >= 0 and s_offset >= 0 and q_offset < len(query) and s_offset < len(sequence) and query[q_offset] == sequence[s_offset]:
            count += 1
            matches += 1 if query[q_offset] == sequence[s_offset] else 0
            q_offset += direction
            s_offset += direction
        return q_offset - direction, s_offset - direction, matches, count

    def best_hit(self, query, sequence, w):
        query_map = self.query_map(query)
        hits = self.hits(query_map, sequence)
        best_score = -1
        best_hit = None
        for hit in hits:
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, hit[0], hit[1], 1)
            o1_start, o2_start = o1 - count + 1, o2 - count + 1
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, o1_start, o2_start, -1)
            score = matches / count
            if score > best_score or (score == best_score and count < best_hit[2]):
                best_score = score
                best_hit = (o1_start, o2_start, count, matches)
        return best_hit

In [55]:
query = "AATATAT"
seq = "AATATGTTATATAATAATATTT"
w = 3
blast = Blast(w, seq)
# result = blast.hits(blast.query_map(query), seq)
# print(result)
results = blast.best_hit(query, seq, w)
print(results)

(0, 0, 1, 1)


In [73]:
query = "TACCTTG"
w = 3
for i in sequences:
    seq=''.join(i)
    blast = Blast(w, seq)
    results = blast.best_hit(query, seq, w)
    print(results)

(0, 0, 1, 1)
None
(1, 11, 1, 1)
(2, 17, 1, 1)
None
(0, 10, 1, 1)
(2, 16, 1, 1)
(0, 23, 1, 1)
(0, 45, 1, 1)
None
None
(3, 29, 1, 1)
(0, 27, 1, 1)
(1, 35, 1, 1)
(2, 17, 1, 1)


In [5]:
def search_db(query):
    cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG")
    cursor = cnx.cursor()
    cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
    sequences = cursor.fetchall()
    print(sequences)
    cursor.close()
    return sequences
search_db("AA")

[('TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG',), ('CAACACGGGAAACCTCACCC',), ('GGGATGGAAGATGGCGGCCTA',), ('TCACATGGAGTACCACTAAGTTATTTGTAATATTTAATAAAGGAAATCTC',), ('GTTTGGTGAACTTCTGCCTA',), ('CCAAGAACAAATAACTTGTAAAATACACTGTTTTTTCACTGCAAATGTGT',), ('TGTGAATGAAAAAAGCAGCCAGTCAGTAGGAAACACTGTCTTGTATACAC',), ('TTTTTTTTTTTTTTTAACAATGGAGAAATCCATCAAAACATGGGTCATTT',), ('TTTTTTAAGTCAGTAAAAAGTAGGCACATCTTTATTTGGCAATGTCATAT',), ('GAATTCGTGCGCAACGCAATCCGGAAGTACAAGCAACTGCAGAAGCTGGT',), ('GGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC',), ('NNNNNNNAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTA',)]


[('TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG',),
 ('CAACACGGGAAACCTCACCC',),
 ('GGGATGGAAGATGGCGGCCTA',),
 ('TCACATGGAGTACCACTAAGTTATTTGTAATATTTAATAAAGGAAATCTC',),
 ('GTTTGGTGAACTTCTGCCTA',),
 ('CCAAGAACAAATAACTTGTAAAATACACTGTTTTTTCACTGCAAATGTGT',),
 ('TGTGAATGAAAAAAGCAGCCAGTCAGTAGGAAACACTGTCTTGTATACAC',),
 ('TTTTTTTTTTTTTTTAACAATGGAGAAATCCATCAAAACATGGGTCATTT',),
 ('TTTTTTAAGTCAGTAAAAAGTAGGCACATCTTTATTTGGCAATGTCATAT',),
 ('GAATTCGTGCGCAACGCAATCCGGAAGTACAAGCAACTGCAGAAGCTGGT',),
 ('GGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC',),
 ('NNNNNNNAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTA',)]